가치 반복법 구현

In [8]:
import numpy as np

In [9]:
ACTIONS = ('U','D','L','R')  # 격자 세계에서 할 수 있는 행동 선언
DELTA_THRESHOLD = 1e-3
GAMMA = 0.9 # 할인율

In [10]:
# 격자 공간의 클래스를 정의
class Grid:
    def __init__(self, rows, cols, start):
        self.rows = rows # 격자 세계의 가로
        self.cols = cols # 격자 세계의 세로
        self.i = start[0] # 시작 좌표의 x
        self.j = start[1] # 시작 좌표의 y
        
    def set(self, rewards, actions): # 보상, 행동 생성자
        self.rewards = rewards # 보상 생성
        self.actions = actions # 행동 생성
    
    def set_state(self, s):
        self.i = s[0]
        self.j = s[1]
        
    def current_state(self):
        return (self.i , self.j)

    def is_terminal(self, s):
        return (s not in self.actions)
    
    def move(self, action):
        if action in self.actions[(self.i , self.j)]:
            if action == 'U':
                self.i -= 1
            elif action == 'D':
                self.i += 1
            elif action == 'R':
                self.j += 1
            elif action == 'L':
                self.j -= 1
        
        return self.rewards.get((self.i , self.j), 0)
    
    def all_states(self):
        return (set(self.actions.keys()) | set(self.rewards.keys())) # 확실히 모르겠음

In [11]:
# 격자 공간과 각 상태에서 선택 가능한 행동을 정의
def standard_grid():
    grid = Grid(3,4,(2,0)) # Grid 객체 매개변수 : 가로, 세로, 시작 좌표
    rewards = {(0,3): 1, (1,3): -1} # 가치 선언, 0인 가치는 작성하지 않음
    actions = {
        (0,0) : ('D','R'),
        (0,1) : ('L','R'),
        (0,2) : ('L','D','R'),
        (1,0) : ('U','D'),
        # (1,1) 을 없는 공간이므로 선언하지 않음
        (1,2) : ('U','D','R'),
        (2,0) : ('U','R'),
        (2,1) : ('L','R'),
        (2,2) : ('L','R','U'),
        (2,3) : ('L','U'),
    } # 행동 선언
    grid.set(rewards, actions) # 보상, 행동 생성자
    return grid # 생성한 그리드 객체 반환

In [12]:
def print_values(V, grid): # 초기 보상 출력 함수
    for i in range(grid.rows): # 가로 반복
        print("-------------------------") # 가로 이미지 출력
        for j in range(grid.cols): # 세로 반복
            value = V.get((i,j), 0) # dic.get() : 딕셔너리의 (i,j)를 찾고 없으면 0을 반환
            # if value >= 0:
            #     print("%.2f | " % value, end="")
            # else:
            #     print("%.2f | " % value, end="")
            print("%.2f | " % value, end="")
            
        print("")

In [13]:
def print_policy(P, grid): # 초기 정책 출력 함수
    for i in range(grid.rows):
        print("-------------------------")
        for j in range(grid.cols):
            action = P.get((i,j), ' ')
            print("  %s  |" % action, end="")
        print("")

In [14]:
if __name__ == '__main__':
    grid = standard_grid() # 그리드 생성 함수: 함수 안에 모든 옵션을 작성함
    
    print("\n보상: ")
    print_values(grid.rewards, grid) # 매개변수로 grid의 보상변수, grid 객체 전달
    
    policy = {} # 정책 딕셔너리 초기화
    for s in grid.actions.keys(): # 행동의 key값 리스트
        policy[s] = np.random.choice(ACTIONS) # 4가지 행동 중 랜덤 1개 선택
        
    print("\n초기 정책: ")
    print_policy(policy, grid) # 매개변수로 랜덤한 행동 정책, grid 객체 전달
    
    V = {} # 가치 딕셔너리 초기화
    states = grid.all_states()
    for s in states:
        if s in grid.actions:
            V[s] = np.random.random()
        else:
            V[s] = 0
            
    i = 0
    while True:
        maxChange = 0
        for s in states:
            oldValue = V[s]
            
            if s in policy:
                newValue = float('-inf')
                for a in ACTIONS:
                    grid.set_state(s)
                    r = grid.move(a)
                    v = r + GAMMA * V[grid.current_state()]
                    if v > newValue:
                        newValue = v
                V[s] = newValue
                maxChange = max(maxChange, np.abs(oldValue - V[s]))
        print("\n%i  번째 반복" % i, end="\n")
        print_values(V, grid)
        i += 1
        
        if maxChange < DELTA_THRESHOLD:
            break
    
    for s in policy.keys():
        bestAction = None
        bestValue = float('-inf')
        for a in ACTIONS:
            grid.set_state(s)
            r = grid.move(a)
            v = r + GAMMA * V[grid.current_state()]
            if v > bestValue:
                bestValue = v
                bestAction = a
        policy[s] = bestAction
        
    print("\n가치 함수: ")
    print_values(V, grid)
    
    print("\n정책: ")
    print_policy(policy, grid)


보상: 
-------------------------
0.00 | 0.00 | 0.00 | 1.00 | 
-------------------------
0.00 | 0.00 | 0.00 | -1.00 | 
-------------------------
0.00 | 0.00 | 0.00 | 0.00 | 

초기 정책: 
-------------------------
  R  |  L  |  U  |     |
-------------------------
  U  |     |  D  |     |
-------------------------
  L  |  L  |  U  |  L  |

0  번째 반복
-------------------------
0.77 | 0.46 | 1.00 | 0.00 | 
-------------------------
0.77 | 0.00 | 0.64 | 0.00 | 
-------------------------
0.77 | 0.83 | 0.75 | 0.64 | 

1  번째 반복
-------------------------
0.81 | 0.90 | 1.00 | 0.00 | 
-------------------------
0.73 | 0.00 | 0.90 | 0.00 | 
-------------------------
0.69 | 0.75 | 0.81 | 0.67 | 

2  번째 반복
-------------------------
0.81 | 0.90 | 1.00 | 0.00 | 
-------------------------
0.73 | 0.00 | 0.90 | 0.00 | 
-------------------------
0.66 | 0.73 | 0.81 | 0.73 | 

3  번째 반복
-------------------------
0.81 | 0.90 | 1.00 | 0.00 | 
-------------------------
0.73 | 0.00 | 0.90 | 0.00 | 
---------------------